# Download Active Listings and Related Shop Info
Etsy API limits the offset to 12,000 listings, so that is the maximum number of recently updated listings that can be accessed. (Requested 100 at a time because each request is limited to 100 records)

In [2]:
import requests
import pandas as pd
import json

In [4]:
? requests

### Get all listings that match keywords "stoneware" and "handmade"

In [2]:
#GetListings URL and Parameters for the API
url = "https://openapi.etsy.com/v3/application/listings/active"
key = "r8ymjsnfohuidr1wev7e4cfg"
keyword = "stoneware,handmade"

In [3]:
#initialize the dataframes
dataframe = pd.DataFrame()
df = pd.DataFrame()

In [4]:
#loop that increases the offset by 100 each time due to the API limitation of downloading a maximum of 100 listings at a time
limit = 100
offset = 0
for i in range (1,121):
    parameters = {"client_id":key, "limit":limit, "offset":offset, "keywords":keyword}
    download = requests.get(url, params=parameters)
    data = download.json()
    if "error" in data.keys():
        print("Maximum Offset Reached")
    else:
        dataframe = pd.json_normalize(data['results'])
        df = df.append(dataframe, ignore_index = True)
    offset = offset + limit
    i = i+1
    

In [5]:
#confirm that there are 12,000 rows
len(df)

12000

In [6]:
#OPTIONAL
#write the data to a csv file
#df.to_csv('listingsStonewareHandmade041022.csv')

## Download Shop Info for all the Etsy listings in the data



In [8]:
#NOTE: DELETE LATER You dont need this if you are doing downloading the 12,000 listings and the shop info at the same time
#df = pd.read_csv('12000listingsStonewareHandmade032222.csv')

In [9]:
#Get all the unique shop IDs
shop_ids = df["shop_id"].tolist()
shop_list = list(set(shop_ids))

In [10]:
? shop_list

In [11]:
#initialize the dataframes
dataframeShop = pd.DataFrame()
dfShop = pd.DataFrame()

# Get Shop Info
This loop sends a request to the ETSY API to get the shop info (ratings, number of favorers, etc) for each shop that appeared in the previous dataset

In [12]:
#loop that queries the API for each unique shop ID
limit = 100

for j in range(0, len(shop_list)):
    shop_id = str(shop_list[j])
    url = "https://openapi.etsy.com/v3/application/shops/" + shop_id
    parameters = {"client_id":key, "limit":limit}
    download = requests.get(url, params=parameters)
    dataShops = download.json()
    dataframeShop = pd.json_normalize(dataShops)
    dfShop = dfShop.append(dataframeShop, ignore_index = True)

In [14]:
#OPTIONAL
#write the data to a csv file
#dfShop.to_csv('ShopInfo041022.csv')

## Merge the data

In [15]:
#create a dataframe with just the columns I'm interested in for the listings data
listings = df[['listing_id', 'shop_id', 'title',
       'description', 'num_favorers',
       'listing_type', 'tags', 'materials',
       'style','taxonomy_id','price.amount',
       'price.divisor', 'price.currency_code','creation_timestamp', 'ending_timestamp',
       'original_creation_timestamp','quantity']]

In [16]:
#create a dataframe with just the columns I'm interested in for the shops data
shops = dfShop[['shop_id', 'shop_name', 'create_date', 'title',
       'announcement', 'currency_code', 'is_vacation', 'vacation_message',
       'sale_message',
       'listing_active_count',
       'accepts_custom_requests', 'url',
       'num_favorers', 'languages', 'is_shop_us_based', 'transaction_sold_count',
       'shipping_from_country_iso', 'shop_location_country_iso',
       'review_average', 'review_count']]

In [17]:
#merge the two datasets so that the shop information is added to each listing
data = listings.merge(shops, on="shop_id")

In [18]:
# save the merged data
#NOTE: change the date each time (maybe automate the filename with the correct date for the future)

#data.to_csv('mergedData041022.csv')